# Verarbeitung der Rohdaten
 

## Laden der benötigten Bibliotheken

In [ ]:
# Module installieren
!pip install xarray
!pip install pyproj

     |████████████████████████████████| 6.3 MB 13.6 MB/s 


In [ ]:
# Importe definieren
import numpy as np
import pandas as pd
import xarray as xr
import os
import google.colab
from google.colab import drive
import requests
import pyproj
from pyproj import CRS, Transformer
import csv

print(f'numpy {np.__version__}')
print(f'pandas {pd.__version__}')
print(f'xarray {xr.__version__}')
print(f'requests {requests.__version__}')
print(f'google.colab {google.colab.__version__}')
print(f'pyproj {pyproj.__version__}')
print(f'csv {csv.__version__}')

# numpy 1.21.6
# pandas 1.3.5
# xarray 0.18.2
# google.colab 0.0.1a2
# pyproj 3.2.1
# csv 1.0

numpy 1.21.6
pandas 1.3.5
xarray 0.18.2
requests 2.23.0
google.colab 0.0.1a2
pyproj 3.2.1
csv 1.0


## Option Google Drive 

Option um die Speicherung der Daten auf Google Drive zu deaktivieren
Standart => Google Drive aktiv (google_drive = True)

In [ ]:
# Google Drive für Dateizugriff mounten
google_drive = True

if google_drive:
  drive.mount('/content/drive', force_remount=True)
  gdriveroot = '/content/drive/MyDrive/'
else:
  # Gewünschten Pfad setzen wenn Google Drive NICHT verwendet wird
  gdriveroot = ''


meinpfad = gdriveroot + 'ScriptingProjekt/data/'

Mounted at /content/drive
 1.Semester		     Heizgradtage
 2.Semester		     Motivationsschreiben_SwissRe_Graduate.gdoc
'3. Semester'		     Nützliches
'Colab Notebooks'	     PMEA
 ELHL-FTH		     ScriptingProjekt
 Erfassung_FB_Adliswil.xls   Vorlage_Erfassung_Dietikon_Rupp.xls


## Gemeindedaten

In [ ]:
# Gemeinden der Schweiz und FL zusammenführen 
gmd_pfad = meinpfad + 'source/gmd/'
df_gmd_ch = pd.DataFrame(pd.read_csv(gmd_pfad + '/txt/g1g22.csv', encoding='ISO-8859–1', delimiter=";"))
df_gmd_li = pd.DataFrame(pd.read_csv(gmd_pfad + '/txt/g1g22_li.csv', encoding='ISO-8859–1', delimiter=";"))
df_gmd = pd.concat([df_gmd_ch, df_gmd_li])

# Entferne Duplikate von Gemeindenamen
df_gmd.drop_duplicates(subset ="GMDNAME",keep = 'first', inplace = True)
df_gmd.head()

# Dataframe mit allen Gemeinden speichern
# df_gmd.to_csv(gmd_pfad + 'gmd.csv', sep=';', index=False, encoding='utf8')

,GMDHISTID,GMDNR,GMDNAME,BZHISTID,BZNR,KTNR,GRNR,AREA_HA,E_MIN,E_MAX,N_MIN,N_MAX,E_CNTR,N_CNTR,Z_MIN,Z_MAX,Z_AVG,Z_MED,Z_CNTR
0,13256.0,1,Aeugst am Albis,10053.0,101,1,4,791,2678111,2681155,1234561,1238544,2679300,1235700,533,887,685,673,700
1,11742.0,2,Affoltern am Albis,10053.0,101,1,4,1059,2673826,2678584,1235208,1239338,2676800,1236800,440,749,529,502,490
2,11801.0,3,Bonstetten,10053.0,101,1,4,743,2675745,2679642,1238997,1243159,2677800,1241000,502,715,581,583,544
3,11992.0,4,Hausen am Albis,10053.0,101,1,4,1360,2680394,2686463,1230185,1236411,2682900,1233100,524,915,673,653,610
4,12249.0,5,Hedingen,10053.0,101,1,4,653,2674866,2678972,1238053,1241034,2676400,1239000,475,745,560,543,503


In [ ]:
# Rechne LV95 Format nach WGS84 Format um
# df_gmd = pd.DataFrame(pd.read_csv(gmd_pfad + 'gmd.csv', delimiter=";", encoding='utf8'))

# erstelle leeres DataFrame
df_gmd_ready = pd.DataFrame()

# Füge Gemeinde Namen und Areal fläche (hektar) aus gemDat
# (vorherigem DataFrame) hinzu
df_gmd_ready['GMDNAME'] = df_gmd['GMDNAME']
df_gmd_ready['AREA_HA'] = df_gmd['AREA_HA']

# Koordinaten transferieren von LV95 zu WGS84 Decimal System
# "Um die LV95-Koordinaten im System CH1903+ von den alten LV03-Koordinaten im System 1903 unterscheiden 
# zu können, wird den bisherigen 6-stelligen Koordinaten nun eine siebte Zahl vorangestellt: 
# In der Nord-Süd-Richtung ist es eine 1, in der West-Ost-Richtung eine 2. Zudem werden die Koordinatenachsen 
# neu mit E (Ost-West) und N (Nord-Süd) bezeichnet. In den Landeskarten ändert sich nur die Beschriftung der 
# Koordinaten am Kartenrand."
# Quelle: https://www.swisstopo.admin.ch/de/wissen-fakten/geodaesie-vermessung/bezugsrahmen/lokal/lv95.html

inProj = CRS("EPSG:21781")
outProj = CRS("WGS84")

transformer = Transformer.from_crs(inProj, outProj)

offsetE = 2000000
offsetN = 1000000

df_gmd['E_CNTR'] = df_gmd['E_CNTR'] - offsetE
df_gmd['N_CNTR'] = df_gmd['N_CNTR'] - offsetN
df_gmd['E_MIN'] = df_gmd['E_MIN'] - offsetE
df_gmd['N_MIN'] = df_gmd['N_MIN'] - offsetN
df_gmd['E_MAX'] = df_gmd['E_MAX'] - offsetE
df_gmd['N_MAX'] = df_gmd['N_MAX'] - offsetN


df_gmd_ready['E_CNTR'] , df_gmd_ready['N_CNTR'] = transformer.transform(df_gmd['E_CNTR'].tolist(), df_gmd['N_CNTR'].tolist())
df_gmd_ready['E_MIN'] , df_gmd_ready['N_MIN'] = transformer.transform(df_gmd['E_MIN'].tolist(), df_gmd['N_MIN'].tolist())
df_gmd_ready['E_MAX'] , df_gmd_ready['N_MAX'] = transformer.transform(df_gmd['E_MAX'].tolist(), df_gmd['N_MAX'].tolist())

# Höhenwerte Gemeinde ergänzen
df_gmd_ready['Z_CNTR'] = df_gmd['Z_CNTR']
df_gmd_ready['Z_MIN'] = df_gmd['Z_MIN']
df_gmd_ready['Z_MAX'] = df_gmd['Z_MAX']
df_gmd_ready['Z_DIFF'] = df_gmd['Z_MAX'] - df_gmd['Z_MIN']

print(df_gmd_ready.head())

# Speichern der fertigen Gemeindeliste mit Name, Fläche, Koordinaten und Höhe  
df_gmd_ready.to_csv(meinpfad+'ready/df_gmd.csv', sep=';', index=False, encoding="utf8")

              GMDNAME  AREA_HA     E_CNTR    N_CNTR      E_MIN     N_MIN  \
0     Aeugst am Albis      791  47.267442  8.486527  47.257340  8.470619   
1  Affoltern am Albis     1059  47.277630  8.453683  47.263652  8.414118   
2          Bonstetten      743  47.315287  8.467626  47.297512  8.440111   
3     Hausen am Albis     1360  47.243616  8.533613  47.217709  8.499996   
4            Hedingen      653  47.297463  8.448771  47.289122  8.428333   

       E_MAX     N_MAX  Z_CNTR  Z_MIN  Z_MAX  Z_DIFF  
0  47.292794  8.511550     700    533    887     354  
1  47.300246  8.477703     490    440    749     309  
2  47.334484  8.492369     544    502    715     213  
3  47.272937  8.581306     610    524    915     391  
4  47.315453  8.483131     503    475    745     270  


## Aufbereiten der Referenz Heizgradtage des Hauseigentümerverbandes

In [ ]:
# Einlesen der Rohdaten
dateiname = 'source/HGT_raw.csv'
df_Ref_HGT = pd.read_csv(meinpfad + dateiname,
                 sep=',',
                 encoding='utf-8')
# Anzeigen der ersten Zeilen
df_Ref_HGT.head()

,Station,1981 – 2010 (gem. MeteoSchweiz),2001 – 2010 (gem. SIA Norm 381/3),2011 – 2020 (gem. SIA Norm 381/3)
0,Adelboden,-,-,4338
1,Aigle,-,-,2900
2,Altdorf,-,3073,2951
3,Basel-Binningen,3035,2932,2745
4,Bern / Zollikofen,3526,3441,3300


In [ ]:
# Identifizieren von nichtbenötigten Spalten
del_columns = df_Ref_HGT.columns[df_Ref_HGT.columns.str.startswith(('1981','2001'))]
# Löschen der nichbenötigten Spalten
df_Ref_HGT = df_Ref_HGT.drop(del_columns, axis = 1)
# Umbennenen der Spalte mit den Heizgradtagen
df_Ref_HGT.rename(columns = {'2011 – 2020 (gem. SIA Norm 381/3)':'Ref_HGT_2011_2020'}, inplace = True)
# Anzeigen der ersten Zeilen
df_Ref_HGT.head()

,Station,Ref_HGT_2011_2020
0,Adelboden,4338
1,Aigle,2900
2,Altdorf,2951
3,Basel-Binningen,2745
4,Bern / Zollikofen,3300


In [ ]:
# Überprüfen ob Nulls vorhanden sind
df_Ref_HGT.isnull().values.any()
# False --> Keine NULLs vorhanden

False

In [ ]:
# Speichern der bereinigten Tabelle
dateiname = 'ready/Ref_HGT_HEV.csv'
df_Ref_HGT.to_csv(meinpfad + dateiname, index=False)

## Referenzdatensatz vom HEV anreichern

Der Referenzdatensatz mit dem langjährigen Mittel der Heizgradtage (HGT) vom Hauseigentümerverband (HEV) wird mit den Koordinaten und weiteren Informationen der jeweiligen Wetterstationen ergänzt. Ziel ist, dass der Refernzdatensatz auf einer Karte dargestellt werden kann.

In [ ]:
# Datensatz von MeteoSchweiz in ein DataFrame laden
dateiname = 'source/ch.meteoschweiz.messnetz-automatisch_de.csv'
df_stationen = pd.read_csv(meinpfad + dateiname, delimiter=';', encoding='cp1252')
df_stationen.head()

,Station,Abk.,WIGOS-ID,Stationstyp,Eigentümer,Daten seit,Stationshöhe m. ü. M.,Barometerhöhe m. ü. Boden,KoordinatenE,KoordinatenN,Breitengrad,Längengrad,Kanton,Messungen,Link
0,Aadorf / Tänikon,TAE,0-20000-0-06679,Wetterstation,MeteoSchweiz,01.01.1970,539.0,539.0,2710518.0,1259824.0,47.479892,8.904928,TG,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
1,Aarberg,ABE,0-756-0-ABE,Niederschlagsstation,MeteoSchweiz,01.10.1900,444.0,NaN,2588354.0,1211894.0,47.057969,7.285350,BE,"Niederschlag, Schnee",https://www.meteoschweiz.admin.ch/home/messwer...
2,Acquarossa / Comprovasco,COM,0-20000-0-06756,Wetterstation,MeteoSchweiz,01.10.1900,575.0,576.0,2714985.0,1146451.0,46.459517,8.935486,TI,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
3,Adelboden,ABO,0-20000-0-06735,Wetterstation,"MeteoSchweiz, SLF",01.01.1901,1321.0,1326.0,2609372.0,1148939.0,46.491703,7.560703,BE,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
4,Affoltern i. E.,AIE,0-756-0-AIE,Niederschlagsstation,MeteoSchweiz,01.06.1864,755.0,NaN,2622940.0,1212500.0,47.063125,7.740614,BE,Niederschlag,https://www.meteoschweiz.admin.ch/home/messwer...


In [ ]:
# Datensatz vom HEV in ein DataFrame laden
df_hev = pd.read_csv(meinpfad + '/ready/Ref_HGT_HEV.csv', delimiter=',', index_col=['Station'])
df_hev.head()

,Ref_HGT_2011_2020
Station,
Adelboden,4338
Aigle,2900
Altdorf,2951
Basel-Binningen,2745
Bern / Zollikofen,3300


In [ ]:
# Datensatz vom HEV mit MeteoSchweiz zusammenführen
# HEV mit Referenzspalte zu MeteoSchweiz ergänzen
# und ungleiche Werte manuell anpassen
df_hev['Station_MAP'] = df_hev.index
df_hev.loc['Basel-Binningen','Station_MAP'] = 'Basel / Binningen'
df_hev.loc['Buchs-Aarau','Station_MAP']= 'Buchs / Aarau'
df_hev.loc['Disentis / Sedrun','Station_MAP']= 'Disentis'
df_hev.loc['Col du Gd-St-Bernard','Station_MAP'] = 'Col du Grand St-Bernard'
df_hev.loc['Genève-Cointrin','Station_MAP']= 'Genève / Cointrin'
df_hev.loc['Gütsch ob Andermatt','Station_MAP']= 'Gütsch, Andermatt'
df_hev.loc['Jungfraujoch Sphinx','Station_MAP'] = 'Jungfraujoch'
df_hev.loc['Samedan-Flugplatz','Station_MAP']= 'Samedan'
df_hev.loc['San Bernardino','Station_MAP']= 'S. Bernardino'

# DataFrames verknüpfen über die Spalte Station_MAP und Station
df_hevreferenz = ''
df_hevreferenz = pd.merge(
    left=df_hev,
    right=df_stationen,
    left_on='Station_MAP',
    right_on='Station',
    how='left',
)

# Index auf Spalte Station_MAP setzen
df_hevreferenz.set_index(['Station_MAP'], inplace=True)

# Prüfung auf NaN Werte (Soll = 0)
Anzahl_Mapping_Fehler = df_hevreferenz['Station'].isnull().sum()
if Anzahl_Mapping_Fehler > 0 :
  # Ausgabe der Liste, für alle Einträge mit True muss ein
  # manuelles Mapping erfasst werden, siehe oben ungleiche Werte
  print(df_hevreferenz['Station'].isnull())

# Ergebnis: Neues DataFrame aus den HEV Daten ergänzt mit den Details 
# der Wetterstationen von MeteoSchweiz

# Neues DataFrame HEV Referenz speichern
df_hevreferenz.to_csv(meinpfad + 'ready/hevreferenz.csv')
df_hevreferenz.head()

,Ref_HGT_2011_2020,Station,Abk.,WIGOS-ID,Stationstyp,Eigentümer,Daten seit,Stationshöhe m. ü. M.,Barometerhöhe m. ü. Boden,KoordinatenE,KoordinatenN,Breitengrad,Längengrad,Kanton,Messungen,Link
Station_MAP,,,,,,,,,,,,,,,,
Adelboden,4338,Adelboden,ABO,0-20000-0-06735,Wetterstation,"MeteoSchweiz, SLF",01.01.1901,1321.0,1326.0,2609372.0,1148939.0,46.491703,7.560703,BE,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
Aigle,2900,Aigle,AIG,0-20000-0-06712,Wetterstation,MeteoSchweiz,01.01.1981,381.0,382.0,2560404.0,1130713.0,46.326647,6.924472,VD,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
Altdorf,2951,Altdorf,ALT,0-20000-0-06672,Wetterstation,MeteoSchweiz,01.01.1864,438.0,440.0,2690181.0,1193564.0,46.887069,8.621894,UR,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
Basel / Binningen,2745,Basel / Binningen,BAS,0-20000-0-06601,Wetterstation,MeteoSchweiz,01.01.1755,316.0,318.0,2610909.0,1265612.0,47.541142,7.583525,BL,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...
Bern / Zollikofen,3300,Bern / Zollikofen,BER,0-20000-0-06631,Wetterstation,MeteoSchweiz,01.01.1864,553.0,554.0,2601934.0,1204410.0,46.990744,7.464061,BE,"Temperatur, Feuchte, Taupunkt, Niederschlag, D...",https://www.meteoschweiz.admin.ch/home/messwer...


## Verarbeitung der Temperaturnormwerte (Temperaturrasterdaten (.NC-Data))

In [ ]:
# Lese Rasterdaten aus dem Source Ordner und erstelle pandas Dataframe daraus
temperaturFile = meinpfad + 'source/tnorm9120/tnorm9120/TnormM9120_ch01r.swiss.lv95_000001010000_000012010000.nc'
ds = xr.open_dataset(temperaturFile, decode_times=False )
df_Tnorm = ds.to_dataframe()
df_Tnorm = df_Tnorm.reset_index()
# Wähle die benötigten Spalten aus.
df_Tnorm = df_Tnorm[['time','TnormM9120', 'lon', 'lat']]
# Lösche alle NAs
df_Tnorm = df_Tnorm.dropna()

In [ ]:
# Entferne Duplikate
df_Tnorm = df_Tnorm.drop_duplicates(subset=['time', 'lon', 'lat'])
df_Tnorm.head()

,time,TnormM9120,lon,lat
52872,0.0,1.951909,5.931808,46.108936
52873,1.0,2.723355,5.931808,46.108936
52874,2.0,6.523786,5.931808,46.108936
52875,3.0,10.244779,5.931808,46.108936
52876,4.0,14.248069,5.931808,46.108936
...,...,...,...,...
2081887,7.0,9.312616,10.514886,46.635185
2081888,8.0,5.310950,10.514886,46.635185
2081889,9.0,1.415752,10.514886,46.635185
2081890,10.0,-3.736551,10.514886,46.635185


In [ ]:
# Lade Gemeinde Koordinaten

In [ ]:
gmd = df_gmd_ready[['GMDNAME', 'E_CNTR', 'N_CNTR', 'Z_CNTR', 'Z_DIFF', 'AREA_HA']]

,GMDNAME,E_CNTR,N_CNTR,Z_CNTR,Z_DIFF,AREA_HA
0,Aeugst am Albis,47.267442,8.486527,700,354,791
1,Affoltern am Albis,47.277630,8.453683,490,309,1059
2,Bonstetten,47.315287,8.467626,544,213,743
3,Hausen am Albis,47.243616,8.533613,610,391,1360
4,Hedingen,47.297463,8.448771,503,270,653
...,...,...,...,...,...,...
6,Eschen,47.199037,9.541173,445,1223,1038
7,Mauren,47.215854,9.556361,439,1161,749
8,Gamprin,47.211419,9.503366,462,888,619
9,Ruggell,47.247759,9.533841,431,187,738


In [ ]:
# Berechne für jede Gemeinde ein Koordinaten Fenster.

In [ ]:
# Definiere Fenster Breite und Höhe
fenster_lat = 0.015
fenster_lon = 0.02

# Erstelle vier Spalten, welche für jede Gemeinde die Ecken des Fensters sind
gmd['fenster_lat_1'] = gmd.E_CNTR - fenster_lat/2
gmd['fenster_lat_2'] = gmd.E_CNTR + fenster_lat/2
gmd['fenster_lon_1'] = gmd.N_CNTR - fenster_lon/2
gmd['fenster_lon_2'] = gmd.N_CNTR + fenster_lon/2
gmd.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,GMDNAME,E_CNTR,N_CNTR,Z_CNTR,Z_DIFF,AREA_HA,fenster_lat_1,fenster_lat_2,fenster_lon_1,fenster_lon_2
0,Aeugst am Albis,47.267442,8.486527,700,354,791,47.259942,47.274942,8.476527,8.496527
1,Affoltern am Albis,47.277630,8.453683,490,309,1059,47.270130,47.285130,8.443683,8.463683
2,Bonstetten,47.315287,8.467626,544,213,743,47.307787,47.322787,8.457626,8.477626
3,Hausen am Albis,47.243616,8.533613,610,391,1360,47.236116,47.251116,8.523613,8.543613
4,Hedingen,47.297463,8.448771,503,270,653,47.289963,47.304963,8.438771,8.458771


In [ ]:
# Weise jeder Gemeinde Temperaturdaten (aus dem NetCDF Datensatz), welche in dem vorher definierten Fenster sind, zu.

In [ ]:
# Erstelle ein neues Pandas Dataframe
df_gmd_temp_monthly = pd.DataFrame()

# Iteriere durch die Gemeindenamen der Gemeindedaten und weise jeder Gemeinde die Temperaturdaten (aus dem NetCDF Datensatz), welche im vorher definierten Fenster sind, zu.
for x in gmd.GMDNAME:
  temp = df_Tnorm[df_Tnorm["lat"].between(float(gmd.loc[gmd['GMDNAME'] == x]['fenster_lat_1']), float(gmd.loc[gmd['GMDNAME'] == x]['fenster_lat_2']))]
  temp = temp[temp["lon"].between(float(gmd.loc[gmd['GMDNAME'] == x]['fenster_lon_1']), float(gmd.loc[gmd['GMDNAME'] == x]['fenster_lon_2']))]

  # Erstelle neue Spalten, damit das Dataframe auch Gemeindename, Gemeinde Zentrums Höhe enthält.
  temp['GMDNAME'] = x
  temp['GMD_Z_CENTR'] = int(gmd.loc[gmd['GMDNAME'] == x]['Z_CNTR'].drop_duplicates())

  #temp['GMD_Z_DIFF'] = int(gmd.loc[gmd['GMDNAME'] == x]['Z_DIFF'].drop_duplicates())
  #temp['AREA_HA'] = int(gmd.loc[gmd['GMDNAME'] == x]['AREA_HA'].drop_duplicates())

  # Füge das "Temp" Dataframe dem Dataframe "gmd_temp_monthly" hinzu
  df_gmd_temp_monthly = df_gmd_temp_monthly.append(temp)

  # Lösche das "Temp" Dataframe, um beim neuen Schleifendurchgang wieder ein sauberes leeres "Temp" Dataset zu haben
  del temp

df_gmd_temp_monthly.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,time,TnormM9120,lon,lat,GMDNAME,GMD_Z_CENTR
1184904,0.0,0.235900,8.489135,47.265614,Aeugst am Albis,700
1184905,1.0,0.955684,8.489135,47.265614,Aeugst am Albis,700
1184906,2.0,4.795328,8.489135,47.265614,Aeugst am Albis,700
1184907,3.0,8.618056,8.489135,47.265614,Aeugst am Albis,700
1184908,4.0,12.665635,8.489135,47.265614,Aeugst am Albis,700


## Mittelwertbildung der gefunden Nachbar Rasterpunkt Temperaturen und HGT Berechnung.

In [ ]:
# Neues Dataframe mit ausgewählten Spalten aus dem Dataframe df_gmd_temp_monthly
# Monatliche Normtemperaturen der nahen Punkte von den Gemeindezentren
df_tempGmd = df_gmd_temp_monthly[['time', 'TnormM9120', 'GMDNAME', 'GMD_Z_CENTR']]
df_tempGmd.head()

,time,TnormM9120,GMDNAME,GMD_Z_CENTR
1184904,0.0,0.235900,Aeugst am Albis,700
1184905,1.0,0.955684,Aeugst am Albis,700
1184906,2.0,4.795328,Aeugst am Albis,700
1184907,3.0,8.618056,Aeugst am Albis,700
1184908,4.0,12.665635,Aeugst am Albis,700


In [ ]:
# Bilden der monatlichen Mitteltemperatur an den Nachbarpunkten mit Hilfe der Spalten Time und GMDNAME  
df_Temp_GMD_mean = df_tempGmd.groupby(['time','GMDNAME']).mean('TnormM9120')
df_Temp_GMD_mean.head()

TnormM9120  GMD_Z_CENTR
time GMDNAME                           
0.0  Aadorf       0.387445        543.0
     Aarau        1.171741        388.0
     Aarberg      1.069844        451.0
     Aarburg      0.884974        397.0
     Aarwangen    0.610216        437.0

In [ ]:
#Umbennenen der Spalte
df_Temp_GMD_mean.rename(columns = {'TnormM9120':'T_Mean_Monthly'}, inplace = True)
df_Temp_GMD_mean.head()

T_Mean_Monthly  GMD_Z_CENTR
time GMDNAME                               
0.0  Aadorf           0.387445        543.0
     Aarau            1.171741        388.0
     Aarberg          1.069844        451.0
     Aarburg          0.884974        397.0
     Aarwangen        0.610216        437.0

In [ ]:
# Berechnung der Heizgradtage. Vereinfacht wird mit 30 multipliziert (Mittlere Anzahl Tage pro Monat)
df_Temp_GMD_mean['HGT_temp'] = np.where(df_Temp_GMD_mean['T_Mean_Monthly'] < 12, (20-df_Temp_GMD_mean['T_Mean_Monthly'])*30, 0)
df_Temp_GMD_mean.head()

T_Mean_Monthly  GMD_Z_CENTR    HGT_temp
time GMDNAME                                           
0.0  Aadorf           0.387445        543.0  588.376648
     Aarau            1.171741        388.0  564.847778
     Aarberg          1.069844        451.0  567.904724
     Aarburg          0.884974        397.0  573.450806
     Aarwangen        0.610216        437.0  581.693542

In [ ]:
# Speichern des Dataframe in ein CSV-File (Monatliche Mitteltemperaturen)
# dateiname = 'ready/HGT_monthly_Tnorm.csv'
# df_Temp_GMD_mean.to_csv(meinpfad + dateiname, index=True)

In [ ]:
# Bilden der jährlichen Mitteltemperatur mit Hilfe der Spalten GMDNAME und GMD_Z_CENTR.
df_GMD_HGT_yearly = df_Temp_GMD_mean.groupby(['GMDNAME','GMD_Z_CENTR']).sum('HGT_temp')

In [ ]:
df_GMD_HGT_yearly.head()

,,T_Mean_Monthly,HGT_temp
GMDNAME,GMD_Z_CENTR,,
Aadorf,543.0,109.203293,3311.885010
Aarau,388.0,120.023788,3129.584717
Aarberg,451.0,120.297012,3126.973145
Aarburg,397.0,117.184944,3190.322266
Aarwangen,437.0,113.297523,3254.710693
...,...,...,...
Zwieselberg,644.0,107.140472,3366.585938
Zwingen,346.0,122.912048,3041.805176
Zwischbergen,820.0,85.386330,4225.148438


In [ ]:
# Löschen der Spalte mit der Temperatursumme
del_columns = df_GMD_HGT_yearly.columns[df_GMD_HGT_yearly.columns.str.startswith(('T_Mean'))]
df_GMD_HGT_yearly = df_GMD_HGT_yearly.drop(del_columns, axis = 1)
df_GMD_HGT_yearly.head()

,,HGT_temp
GMDNAME,GMD_Z_CENTR,
Aadorf,543.0,3311.885010
Aarau,388.0,3129.584717
Aarberg,451.0,3126.973145
Aarburg,397.0,3190.322266
Aarwangen,437.0,3254.710693


In [ ]:
# Speichern des Dataframe
dateiname = 'ready/GMD_HGT_yearly.csv'
df_GMD_HGT_yearly.to_csv(meinpfad + dateiname, index=True)

## Vergleich HGT - Werte
Verleich zwischen den errechneten HGT-Werten an den Gemeindezentren aus den Temperaturdaten des 2 km Gitters und den Referenz HGT-Werten des Hauseigentümerverbandes.

In [ ]:
# Laden der berechneten Gemeinde Heizgradtage mithilfe der Normtemperatur
dateiname = 'ready/GMD_HGT_yearly.csv'
df_GMD_HGT_yearlycsv = pd.read_csv(meinpfad + dateiname, sep=',')
df_GMD_HGT_yearlycsv.head()

,GMDNAME,GMD_Z_CENTR,HGT_temp
0,Aadorf,543.0,3311.8850
1,Aarau,388.0,3129.5847
2,Aarberg,451.0,3126.9731
3,Aarburg,397.0,3190.3223
4,Aarwangen,437.0,3254.7107


In [ ]:
# HGT-Werte der Gemeinden aufgrund der Temperaturdaten
df_GMD_HGT_yearlycsv.head()

,GMDNAME,GMD_Z_CENTR,HGT_temp
0,Aadorf,543.0,3311.8850
1,Aarau,388.0,3129.5847
2,Aarberg,451.0,3126.9731
3,Aarburg,397.0,3190.3223
4,Aarwangen,437.0,3254.7107


In [ ]:
# Referenz HGT-Werte der Messsationen (Hauseigentümerverband)
df_Ref_HGT.head()

,Station,Ref_HGT_2011_2020
0,Adelboden,4338
1,Aigle,2900
2,Altdorf,2951
3,Basel-Binningen,2745
4,Bern / Zollikofen,3300


In [ ]:
# Überprüfen welche Messstationen nicht als Gemeindename vorhanden sind
values_list = df_GMD_HGT_yearlycsv['GMDNAME']
df_Ref_HGT[~df_Ref_HGT['Station'].isin(values_list)]

,Station,Ref_HGT_2011_2020
2,Altdorf,2951
3,Basel-Binningen,2745
4,Bern / Zollikofen,3300
5,Buchs-Aarau,3047
8,Cimetta,4517
10,Disentis / Sedrun,4072
11,La Dôle,5260
14,Col du Gd-St-Bernard,7060
15,Genève-Cointrin,2781
17,Gütsch ob Andermatt,6478


In [ ]:
# Manuelle Umbenennung der Messstationen damit sie mit einer Gemeinde übereinstimmen
d = {'Altdorf' : 'Altdorf (UR)' ,'Basel-Binningen': 'Basel' , 'Bern / Zollikofen':'Zollikofen' , 'Buchs-Aarau' : 'Buchs (AG)' , 
      'Disentis / Sedrun' : 'Disentis/Mustér' , 'Genève-Cointrin': 'Genève' , 'Locarno / Monti':'Locarno' , 'Montana' : 'Crans-Montana', 
     'Poschiavo / Robbia' : 'Poschiavo' , 'Samedan-Flugplatz' : 'Samedan' 	, 'Aadorf / Tänikon':'Aadorf' , 'Zürich / Kloten':'Kloten' , 'Zürich / Fluntern':'Zürich'}   
df_Ref_HGT = df_Ref_HGT.replace(d)

In [ ]:
# Wiederholung der Überprüfung welche Messstationen nicht als Gemeindename vorhanden sind
values_list = df_GMD_HGT_yearlycsv['GMDNAME']
df_Ref_HGT[~df_Ref_HGT['Station'].isin(values_list)]

,Station,Ref_HGT_2011_2020
8,Cimetta,4517
11,La Dôle,5260
14,Col du Gd-St-Bernard,7060
17,Gütsch ob Andermatt,6478
20,Jungfraujoch Sphinx,9558
24,Le Moléson,5678
26,Napf,4619
29,Pilatus,6065
34,San Bernardino,5166
35,Säntis,7309


In [ ]:
# Es bleiben die Bergstationen übrig, aufgrund er Höhenlage werden die Bergstationen als Ausreisser handgehabt.
# D.h. für den Verlgeich werden sie nicht berücksichtigt.

In [ ]:
# Neues Dataframe für den Vergleich der HGT-Werte
df_Vergleich_HGT = pd.DataFrame()
df_Vergleich_HGT = df_Ref_HGT.merge(df_GMD_HGT_yearlycsv, left_on='Station', right_on='GMDNAME')
df_Vergleich_HGT.head()

,Station,Ref_HGT_2011_2020,GMDNAME,GMD_Z_CENTR,HGT_temp
0,Adelboden,4338,Adelboden,1356.0,4824.1885
1,Aigle,2900,Aigle,409.0,3075.1392
2,Altdorf (UR),2951,Altdorf (UR),460.0,3236.2786
3,Basel,2745,Basel,267.0,2849.7212
4,Zollikofen,3300,Zollikofen,563.0,3304.8875


In [ ]:
# Errechnung der Differenz zwischen den HGT-Referenzwerten und den aus der Temperatur errechneten HGT-Werten
# hgtdiff = Differenz der HGT-Werten
# hgtdiff_Pr = Differenz der HGT-Werten in Prozent
df_Vergleich_HGT['hgtdiff'] = df_Vergleich_HGT.Ref_HGT_2011_2020 - df_Vergleich_HGT.HGT_temp
df_Vergleich_HGT['hgtdiff_Pr'] = df_Vergleich_HGT.hgtdiff * 100 / df_Vergleich_HGT.Ref_HGT_2011_2020
df_Vergleich_HGT = df_Vergleich_HGT.sort_values('hgtdiff_Pr')
df_Vergleich_HGT.head()

,Station,Ref_HGT_2011_2020,GMDNAME,GMD_Z_CENTR,HGT_temp,hgtdiff,hgtdiff_Pr
32,Vaduz,2825,Vaduz,626.0,3241.3447,-416.3447,-14.737865
0,Adelboden,4338,Adelboden,1356.0,4824.1885,-486.1885,-11.207665
17,Lugano,2151,Lugano,276.0,2371.1190,-220.1190,-10.233333
30,Stabio,2616,Stabio,356.0,2878.8694,-262.8694,-10.048524
28,Sion,2830,Sion,512.0,3107.1680,-277.1680,-9.793922


In [ ]:
# Ausgabe der minimalen bzw. maximalen Abweichungen
AbweichungMin = round(df_Vergleich_HGT.hgtdiff_Pr.min(),2)
AbweichungMax = round(df_Vergleich_HGT.hgtdiff_Pr.max(),2)
print(f'Die Abweichungen liegen zwischen {AbweichungMin} und {AbweichungMax} Prozent.')

Die Abweichungen liegen zwischen -14.74 und 4.37 Prozent.


In [ ]:
# Speichern des Dataframe
dateiname = 'ready/Vergleich_HGT.csv'
df_Vergleich_HGT.to_csv(meinpfad + dateiname, index=True)

## Erstelle Dataframe für den geografischen Plot aller Messstationen, Rasterpunkte und Gemeindezentren mit dazugehörigen HGT

In [ ]:
# Bereite Rasterpunkt Daten auf, damit sie später mit den anderen Daten in einem Plot dargestellt werden können
df_rasterpunkte_hgt = df_gmd_temp_monthly[['GMDNAME', 'lat', 'lon']].drop_duplicates()
df_rasterpunkte_hgt['Art'] = 'Rasterpunkt'
df_rasterpunkte_hgt['HGT'] =  'Kein HGT für Rasterpunkte berechnet'
df_rasterpunkte_hgt.head()

,GMDNAME,lat,lon,Art,HGT
1184904,Aeugst am Albis,47.265614,8.489135,Rasterpunkt,Kein HGT für Rasterpunkte berechnet
1184928,Aeugst am Albis,47.274609,8.489311,Rasterpunkt,Kein HGT für Rasterpunkte berechnet
1167648,Affoltern am Albis,47.274963,8.449668,Rasterpunkt,Kein HGT für Rasterpunkte berechnet
1167672,Affoltern am Albis,47.283958,8.449838,Rasterpunkt,Kein HGT für Rasterpunkte berechnet
1173408,Affoltern am Albis,47.274845,8.462882,Rasterpunkt,Kein HGT für Rasterpunkte berechnet


In [ ]:
# Bereite Gemeinde Daten auf, damit sie später mit den anderen Daten zusammengefügt werden können
df_gemeinde = df_gmd_ready[['GMDNAME', 'E_CNTR', 'N_CNTR']]
df_gemeinde = df_gemeinde.rename(columns={"E_CNTR":"lat",'N_CNTR':'lon'})
df_gemeinde['Art'] = 'Gemeinde_Mittelpunkt'
# Joine die berechneten HGT Werte zum Dataset           
df_gemeinde_hgt = df_gemeinde.merge(df_GMD_HGT_yearlycsv, left_on='GMDNAME', right_on='GMDNAME')
df_gemeinde_hgt = df_gemeinde_hgt[['GMDNAME', 'lat', 'lon', 'Art', 'HGT_temp']]
df_gemeinde_hgt = df_gemeinde_hgt.rename(columns={"HGT_temp":"HGT"})
df_gemeinde_hgt.head()

,GMDNAME,lat,lon,Art
0,Aeugst am Albis,47.267442,8.486527,Gemeinde_Mittelpunkt
1,Affoltern am Albis,47.277630,8.453683,Gemeinde_Mittelpunkt
2,Bonstetten,47.315287,8.467626,Gemeinde_Mittelpunkt
3,Hausen am Albis,47.243616,8.533613,Gemeinde_Mittelpunkt
4,Hedingen,47.297463,8.448771,Gemeinde_Mittelpunkt


In [ ]:
# Bereite Wetterstationen Daten auf, damit sie später mit den anderen Daten zusammengefügt werden können
df_wetterstationen_hgt = df_hevreferenz[['Station', 'Breitengrad', 'Längengrad', 'Ref_HGT_2011_2020']]
df_wetterstationen_hgt = df_wetterstationen_hgt.rename(columns={"Breitengrad":"lat",'Längengrad':'lon', 'Station': 'GMDNAME', 'Ref_HGT_2011_2020': 'HGT'})
df_wetterstationen_hgt['Art'] = 'Wetterstation'
df_wetterstationen_hgt.head()

,GMDNAME,lat,lon,HGT,Art
Station_MAP,,,,,
Adelboden,Adelboden,46.491703,7.560703,4338,Wetterstation
Aigle,Aigle,46.326647,6.924472,2900,Wetterstation
Altdorf,Altdorf,46.887069,8.621894,2951,Wetterstation
Basel / Binningen,Basel / Binningen,47.541142,7.583525,2745,Wetterstation
Bern / Zollikofen,Bern / Zollikofen,46.990744,7.464061,3300,Wetterstation


In [ ]:
#Füge diese drei Datasets zu einem Dataset zusammen. Dies wird für den Plot später gebraucht, daher wird es zu einem csv exportiert.
df_map = df_rasterpunkte_hgt.append(df_gemeinde_hgt)
df_map = df_map.append(df_wetterstationen_hgt)
df_map.to_csv(meinpfad + 'ready/df_geo_hgt.csv', index=True)